In [1]:
%matplotlib inline

import sys
from netCDF4 import Dataset
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num
import time as time2
import numpy as np
import pandas as pd
import xarray as xr
import eofs
from eofs.standard import Eof
import glob

# you need intake-esm V 2020.11.4 and intake V 0.6.0

# import tensorflow as tf

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely.geometry.polygon import LinearRing


import matplotlib as mpl
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from matplotlib import cm

import copy
import fsspec
import intake

/ncar/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
# Read Control Run
d = '/glade/scratch/acsubram/S2S_Database/QUV_20170102.grb'
dsctl = xr.open_dataset(d, engine='cfgrib')

In [3]:
dsctl

<xarray.Dataset>
Dimensions:        (isobaricInhPa: 7, latitude: 121, longitude: 240, number: 50, step: 47)
Coordinates:
  * number         (number) int64 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
    time           datetime64[ns] ...
  * step           (step) timedelta64[ns] 0 days 1 days ... 45 days 46 days
  * isobaricInhPa  (isobaricInhPa) int64 1000 925 850 700 500 300 200
  * latitude       (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude      (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time     (step) datetime64[ns] ...
Data variables:
    u              (number, step, isobaricInhPa, latitude, longitude) float32 ...
    v              (number, step, isobaricInhPa, latitude, longitude) float32 ...
    q              (number, step, isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Fore

In [4]:
u = dsctl['u']
v = dsctl['v']
q = dsctl['q']
time_ctl = dsctl['time']
lon_ctl = dsctl['longitude']
lat_ctl = dsctl['latitude']
ens_ctl = dsctl['number']
step_ctl = dsctl['step']
prs_ctl = dsctl['isobaricInhPa']


In [11]:
prs_ctl

<xarray.DataArray 'isobaricInhPa' (isobaricInhPa: 7)>
array([1000,  925,  850,  700,  500,  300,  200])
Coordinates:
    time           datetime64[ns] 2017-01-02
  * isobaricInhPa  (isobaricInhPa) int64 1000 925 850 700 500 300 200
Attributes:
    long_name:         pressure
    units:             hPa
    positive:          down
    stored_direction:  decreasing
    standard_name:     air_pressure

# Compute IVT for each initialized forecast in a loop

In [23]:
import re


for file in glob.glob("QUV*.grb"):
    # Read Control Run
    ds = file
    dsopen = xr.open_dataset(ds, engine='cfgrib')
    u = dsopen['u']
    v = dsopen['v']
    q = dsopen['q']
    time_ctl = dsopen['time']
    lon_ctl = dsopen['longitude']
    lat_ctl = dsopen['latitude']
    ens_ctl = dsopen['number']
    step_ctl = dsopen['step']
    prs_ctl = dsopen['isobaricInhPa']
    # calculate the zonal and meridional horizontal vapour transport
    # Pressure levels: 1000 925 850 700 500 300 200


    #200 - 300mb
    qu0 = np.mean(q.sel(isobaricInhPa=[200,300]),axis=2) * np.mean(u.sel(isobaricInhPa=[200,300]),axis=2)*10000/9.8
    #300 - 500mb
    qu1 = np.mean(q.sel(isobaricInhPa=[300,500]),axis=2) * np.mean(u.sel(isobaricInhPa=[300,500]),axis=2)*20000/9.8
    #500 - 700 mb
    qu2 = np.mean(q.sel(isobaricInhPa=[500,700]),axis=2) * np.mean(u.sel(isobaricInhPa=[500,700]),axis=2)*20000/9.8
    #700 - 850 mb
    qu3 = np.mean(q.sel(isobaricInhPa=[700,850]),axis=2) * np.mean(u.sel(isobaricInhPa=[700,850]),axis=2)*15000/9.8
    #850 - 925 mb
    qu4 = np.mean(q.sel(isobaricInhPa=[850,925]),axis=2) * np.mean(u.sel(isobaricInhPa=[850,925]),axis=2)*7500/9.8
    #925 - 1000 mb
    qu5 = np.mean(q.sel(isobaricInhPa=[925,1000]),axis=2) * np.mean(u.sel(isobaricInhPa=[925,1000]),axis=2)*7500/9.8

    qu = qu0+qu1+qu2+qu3+qu4+qu5


    #200 - 300mb
    qv0 = np.mean(q.sel(isobaricInhPa=[200,300]),axis=2) * np.mean(v.sel(isobaricInhPa=[200,300]),axis=2)*10000/9.8
    #300 - 500mb
    qv1 = np.mean(q.sel(isobaricInhPa=[300,500]),axis=2) * np.mean(v.sel(isobaricInhPa=[300,500]),axis=2)*20000/9.8
    #500 - 700 mb
    qv2 = np.mean(q.sel(isobaricInhPa=[500,700]),axis=2) * np.mean(v.sel(isobaricInhPa=[500,700]),axis=2)*20000/9.8
    #700 - 850 mb
    qv3 = np.mean(q.sel(isobaricInhPa=[700,850]),axis=2) * np.mean(v.sel(isobaricInhPa=[700,850]),axis=2)*15000/9.8
    #850 - 925 mb
    qv4 = np.mean(q.sel(isobaricInhPa=[850,925]),axis=2) * np.mean(v.sel(isobaricInhPa=[850,925]),axis=2)*7500/9.8
    #925 - 1000 mb
    qv5 = np.mean(q.sel(isobaricInhPa=[925,1000]),axis=2) * np.mean(v.sel(isobaricInhPa=[925,1000]),axis=2)*7500/9.8


    qv = qv0+qv1+qv2+qv3+qv4+qv5
    quv = np.sqrt( qu**2 + qv**2 )

    dns = xr.Dataset({'ivt':quv})
    dns['ivt'] = quv
    
    res = re.findall("QUV_(\d+).grb", file)
    if not res: continue
    
    dns.to_netcdf("IVT_"+res[0]+".nc")
    print("IVT_"+res[0]+".nc")
    
    




IVT_20170130.nc
IVT_20170529.nc
IVT_20170424.nc
IVT_20170420.nc
IVT_20170427.nc
IVT_20170320.nc
IVT_20170403.nc
IVT_20170413.nc
IVT_20170119.nc
IVT_20170213.nc
IVT_20170508.nc
IVT_20170209.nc
IVT_20170330.nc
IVT_20170109.nc
IVT_20170501.nc
IVT_20170525.nc
IVT_20170518.nc
IVT_20170511.nc
IVT_20170309.nc
IVT_20170123.nc
IVT_20170116.nc
IVT_20170504.nc
IVT_20170105.nc
IVT_20170223.nc


# Compute IVT for the Reanalysis fields

In [24]:
# Read Control Run
ds = '/glade/scratch/acsubram/S2S_Database/ERA5/ERA5_QUV_20161201_20170531.grb'
dsopen = xr.open_dataset(ds, engine='cfgrib')
u = dsopen['u']
v = dsopen['v']
q = dsopen['q']
time_ctl = dsopen['time'].data
lon_ctl = dsopen['longitude']
lat_ctl = dsopen['latitude']
ens_ctl = dsopen['number']
step_ctl = dsopen['step']
prs_ctl = dsopen['isobaricInhPa']
# calculate the zonal and meridional horizontal vapour transport
# Pressure levels: 1000 925 850 700 500 300 200

In [25]:
dsopen

<xarray.Dataset>
Dimensions:        (isobaricInhPa: 7, latitude: 121, longitude: 240, time: 182)
Coordinates:
    number         int64 ...
  * time           (time) datetime64[ns] 2016-12-01 2016-12-02 ... 2017-05-31
    step           timedelta64[ns] ...
  * isobaricInhPa  (isobaricInhPa) int64 1000 925 850 700 500 300 200
  * latitude       (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude      (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time     (time) datetime64[ns] ...
Data variables:
    q              (time, isobaricInhPa, latitude, longitude) float32 ...
    u              (time, isobaricInhPa, latitude, longitude) float32 ...
    v              (time, isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    instit

In [27]:
import re

#200 - 300mb
qu0 = np.mean(q.sel(isobaricInhPa=[200,300]),axis=1) * np.mean(u.sel(isobaricInhPa=[200,300]),axis=1)*10000/9.8
#300 - 500mb
qu1 = np.mean(q.sel(isobaricInhPa=[300,500]),axis=1) * np.mean(u.sel(isobaricInhPa=[300,500]),axis=1)*20000/9.8
#500 - 700 mb
qu2 = np.mean(q.sel(isobaricInhPa=[500,700]),axis=1) * np.mean(u.sel(isobaricInhPa=[500,700]),axis=1)*20000/9.8
#700 - 850 mb
qu3 = np.mean(q.sel(isobaricInhPa=[700,850]),axis=1) * np.mean(u.sel(isobaricInhPa=[700,850]),axis=1)*15000/9.8
#850 - 925 mb
qu4 = np.mean(q.sel(isobaricInhPa=[850,925]),axis=1) * np.mean(u.sel(isobaricInhPa=[850,925]),axis=1)*7500/9.8
#925 - 1000 mb
qu5 = np.mean(q.sel(isobaricInhPa=[925,1000]),axis=1) * np.mean(u.sel(isobaricInhPa=[925,1000]),axis=1)*7500/9.8

qu = qu0+qu1+qu2+qu3+qu4+qu5


#200 - 300mb
qv0 = np.mean(q.sel(isobaricInhPa=[200,300]),axis=1) * np.mean(v.sel(isobaricInhPa=[200,300]),axis=1)*10000/9.8
#300 - 500mb
qv1 = np.mean(q.sel(isobaricInhPa=[300,500]),axis=1) * np.mean(v.sel(isobaricInhPa=[300,500]),axis=1)*20000/9.8
#500 - 700 mb
qv2 = np.mean(q.sel(isobaricInhPa=[500,700]),axis=1) * np.mean(v.sel(isobaricInhPa=[500,700]),axis=1)*20000/9.8
#700 - 850 mb
qv3 = np.mean(q.sel(isobaricInhPa=[700,850]),axis=1) * np.mean(v.sel(isobaricInhPa=[700,850]),axis=1)*15000/9.8
#850 - 925 mb
qv4 = np.mean(q.sel(isobaricInhPa=[850,925]),axis=1) * np.mean(v.sel(isobaricInhPa=[850,925]),axis=1)*7500/9.8
#925 - 1000 mb
qv5 = np.mean(q.sel(isobaricInhPa=[925,1000]),axis=1) * np.mean(v.sel(isobaricInhPa=[925,1000]),axis=1)*7500/9.8

qv = qv0+qv1+qv2+qv3+qv4+qv5
quv = np.sqrt( qu**2 + qv**2 )

dns = xr.Dataset({'ivt':quv})
dns['ivt'] = quv

dns.to_netcdf("ERA5/ERA5_IVT_2016_2017_time.nc")

In [5]:
import re


for file in glob.glob("ERA5/ERA5_QUV_2017*.grb"):
    # Read Control Run
    des = file
    desopen = xr.open_dataset(des, engine='cfgrib')
    u = desopen['u']
    v = desopen['v']
    q = desopen['q']
    time_ctl = desopen['time']
    lon_ctl = desopen['longitude']
    lat_ctl = desopen['latitude']
    ens_ctl = desopen['number']
    step_ctl = desopen['step']
    prs_ctl = desopen['isobaricInhPa']
    # calculate the zonal and meridional horizontal vapour transport
    # Pressure levels: 1000 925 850 700 500 300 200


    #200 - 300mb
    qu0 = np.mean(q.sel(isobaricInhPa=[200,300]),axis=0) * np.mean(u.sel(isobaricInhPa=[200,300]),axis=0)*10000/9.8
    #300 - 500mb
    qu1 = np.mean(q.sel(isobaricInhPa=[300,500]),axis=0) * np.mean(u.sel(isobaricInhPa=[300,500]),axis=0)*20000/9.8
    #500 - 700 mb
    qu2 = np.mean(q.sel(isobaricInhPa=[500,700]),axis=0) * np.mean(u.sel(isobaricInhPa=[500,700]),axis=0)*20000/9.8
    #700 - 850 mb
    qu3 = np.mean(q.sel(isobaricInhPa=[700,850]),axis=0) * np.mean(u.sel(isobaricInhPa=[700,850]),axis=0)*15000/9.8
    #850 - 925 mb
    qu4 = np.mean(q.sel(isobaricInhPa=[850,925]),axis=0) * np.mean(u.sel(isobaricInhPa=[850,925]),axis=0)*7500/9.8
    #925 - 1000 mb
    qu5 = np.mean(q.sel(isobaricInhPa=[925,1000]),axis=0) * np.mean(u.sel(isobaricInhPa=[925,1000]),axis=0)*7500/9.8

    qu = qu0+qu1+qu2+qu3+qu4+qu5


    #200 - 300mb
    qv0 = np.mean(q.sel(isobaricInhPa=[200,300]),axis=0) * np.mean(v.sel(isobaricInhPa=[200,300]),axis=0)*10000/9.8
    #300 - 500mb
    qv1 = np.mean(q.sel(isobaricInhPa=[300,500]),axis=0) * np.mean(v.sel(isobaricInhPa=[300,500]),axis=0)*20000/9.8
    #500 - 700 mb
    qv2 = np.mean(q.sel(isobaricInhPa=[500,700]),axis=0) * np.mean(v.sel(isobaricInhPa=[500,700]),axis=0)*20000/9.8
    #700 - 850 mb
    qv3 = np.mean(q.sel(isobaricInhPa=[700,850]),axis=0) * np.mean(v.sel(isobaricInhPa=[700,850]),axis=0)*15000/9.8
    #850 - 925 mb
    qv4 = np.mean(q.sel(isobaricInhPa=[850,925]),axis=0) * np.mean(v.sel(isobaricInhPa=[850,925]),axis=0)*7500/9.8
    #925 - 1000 mb
    qv5 = np.mean(q.sel(isobaricInhPa=[925,1000]),axis=0) * np.mean(v.sel(isobaricInhPa=[925,1000]),axis=0)*7500/9.8


    qv = qv0+qv1+qv2+qv3+qv4+qv5
    quv = np.sqrt( qu**2 + qv**2 )

    dnes = xr.Dataset({'ivt':quv})
    dnes['ivt'] = quv
    
    res = re.findall("ERA5/ERA5_QUV_(\d+).grb", file)
    if not res: continue
    
    dnes.to_netcdf("ERA5/ERA5_IVT_"+res[0]+".nc")
    print("ERA5/ERA5_IVT_"+res[0]+".nc")
    
    

ERA5/ERA5_IVT_20170313.nc
ERA5/ERA5_IVT_20170514.nc
ERA5/ERA5_IVT_20170424.nc
ERA5/ERA5_IVT_20170131.nc
ERA5/ERA5_IVT_20170218.nc
ERA5/ERA5_IVT_20170414.nc
ERA5/ERA5_IVT_20170512.nc
ERA5/ERA5_IVT_20170510.nc
ERA5/ERA5_IVT_20170502.nc
ERA5/ERA5_IVT_20170305.nc
ERA5/ERA5_IVT_20170426.nc
ERA5/ERA5_IVT_20170110.nc
ERA5/ERA5_IVT_20170513.nc
ERA5/ERA5_IVT_20170328.nc
ERA5/ERA5_IVT_20170123.nc
ERA5/ERA5_IVT_20170212.nc
ERA5/ERA5_IVT_20170519.nc
ERA5/ERA5_IVT_20170418.nc
ERA5/ERA5_IVT_20170330.nc
ERA5/ERA5_IVT_20170326.nc
ERA5/ERA5_IVT_20170205.nc
ERA5/ERA5_IVT_20170207.nc
ERA5/ERA5_IVT_20170225.nc
ERA5/ERA5_IVT_20170104.nc
ERA5/ERA5_IVT_20170105.nc
ERA5/ERA5_IVT_20170408.nc
ERA5/ERA5_IVT_20170507.nc
ERA5/ERA5_IVT_20170505.nc
ERA5/ERA5_IVT_20170515.nc
ERA5/ERA5_IVT_20170417.nc
ERA5/ERA5_IVT_20170202.nc
ERA5/ERA5_IVT_20170525.nc
ERA5/ERA5_IVT_20170224.nc
ERA5/ERA5_IVT_20170508.nc
ERA5/ERA5_IVT_20170406.nc
ERA5/ERA5_IVT_20170411.nc
ERA5/ERA5_IVT_20170301.nc
ERA5/ERA5_IVT_20170506.nc
ERA5/ERA5_IV